In [1]:
#! pip install -U prettytable

Look at this site <br>
https://www.districtdatalabs.com/exploring-bureau-of-labor-statistics-time-series

In [2]:
import pandas as pd
from prettytable import prettytable
import requests
import json

In [3]:
#api_key = pd.read_csv('data/bls_api.txt')

In [4]:
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ["CUUR0000SA0", "CES3000000006", "JTU300000000000000JOL", 
                                "JTU300000000000000HIL", "JTU300000000000000TSL"],
                   "startyear":"2002",
                   "endyear":"2021", 
                   #"calculations":"true",
                   "registrationkey":"92eed6734bf5495f88972d9ba4106350"
                  })
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
# for series in json_data['Results']['series']:
#     MSeparations=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
#     #MSeparations=pd.DataFrame(["series id","year","period","value","footnotes"])
#     seriesId = series['seriesID']
#     for item in series['data']:
#         year = item['year']
#         period = item['period']
#         value = item['value']
#         footnotes=""
#         for footnote in item['footnotes']:
#             if footnote:
#                 footnotes = footnotes + footnote['text'] + ','
#         if 'M01' <= period <= 'M12':
#             MSeparations.add_row([seriesId,year,period,value,footnotes[0:-1]])
#             #MSeparations.append([seriesId,year,period,value,footnotes[0:- 1]])
#     output = open(seriesId + '.txt','w')
#     output.write (MSeparations.get_string())
#     #output.write (str(MSeparations))
#     output.close()

In [5]:
json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 607,
 'message': [],
 'Results': {'series': [{'seriesID': 'CUUR0000SA0',
    'data': [{'year': '2021',
      'period': 'M04',
      'periodName': 'April',
      'latest': 'true',
      'value': '267.054',
      'footnotes': [{}]},
     {'year': '2021',
      'period': 'M03',
      'periodName': 'March',
      'value': '264.877',
      'footnotes': [{}]},
     {'year': '2021',
      'period': 'M02',
      'periodName': 'February',
      'value': '263.014',
      'footnotes': [{}]},
     {'year': '2021',
      'period': 'M01',
      'periodName': 'January',
      'value': '261.582',
      'footnotes': [{}]},
     {'year': '2020',
      'period': 'M12',
      'periodName': 'December',
      'value': '260.474',
      'footnotes': [{}]},
     {'year': '2020',
      'period': 'M11',
      'periodName': 'November',
      'value': '260.229',
      'footnotes': [{}]},
     {'year': '2020',
      'period': 'M10',
      'periodName': 'October',
   

In [6]:
json_data['Results']['series'][1]['seriesID']

'CES3000000006'

In [7]:
BLS_dfs = []  #set up container to hold df to be built
for i in range(5):  #count 0-5
    BLS_data = pd.DataFrame(json_data['Results']['series'][i]['data']) #builds df out of the i result set
    BLS_data['Report_Name'] = json_data['Results']['series'][i]['seriesID'] #creates new column to identify the SeriesID
    BLS_dfs.append(BLS_data) #store the resulting df in df container     
    
#pd.read_json(json_data['Results']['series']['data'])

In [8]:
BLS_dfs   #list that contains the 5 dataframes created above

[     year period periodName latest    value footnotes  Report_Name
 0    2021    M04      April   true  267.054      [{}]  CUUR0000SA0
 1    2021    M03      March    NaN  264.877      [{}]  CUUR0000SA0
 2    2021    M02   February    NaN  263.014      [{}]  CUUR0000SA0
 3    2021    M01    January    NaN  261.582      [{}]  CUUR0000SA0
 4    2020    M12   December    NaN  260.474      [{}]  CUUR0000SA0
 ..    ...    ...        ...    ...      ...       ...          ...
 227  2002    M05        May    NaN    179.8      [{}]  CUUR0000SA0
 228  2002    M04      April    NaN    179.8      [{}]  CUUR0000SA0
 229  2002    M03      March    NaN    178.8      [{}]  CUUR0000SA0
 230  2002    M02   February    NaN    177.8      [{}]  CUUR0000SA0
 231  2002    M01    January    NaN    177.1      [{}]  CUUR0000SA0
 
 [232 rows x 7 columns],
      year period periodName latest  value  \
 0    2021    M04      April   true   8467   
 1    2021    M03      March    NaN   8514   
 2    2021    M02  

In [9]:
BLS_final = pd.concat(BLS_dfs, ignore_index=True)

In [10]:
BLS_final

,year,period,periodName,latest,value,footnotes,Report_Name
0,2021,M04,April,true,267.054,[{}],CUUR0000SA0
1,2021,M03,March,NaN,264.877,[{}],CUUR0000SA0
2,2021,M02,February,NaN,263.014,[{}],CUUR0000SA0
3,2021,M01,January,NaN,261.582,[{}],CUUR0000SA0
4,2020,M12,December,NaN,260.474,[{}],CUUR0000SA0
...,...,...,...,...,...,...,...
1152,2002,M05,May,NaN,433,[{}],JTU300000000000000TSL
1153,2002,M04,April,NaN,491,[{}],JTU300000000000000TSL
1154,2002,M03,March,NaN,397,[{}],JTU300000000000000TSL
1155,2002,M02,February,NaN,415,[{}],JTU300000000000000TSL


In [11]:
BLS_final = BLS_final.drop(columns = ['period', 'latest']).rename(columns = {'periodName':'month', 'Report_Name':'report'})
     

In [12]:
BLS_final

,year,month,value,footnotes,report
0,2021,April,267.054,[{}],CUUR0000SA0
1,2021,March,264.877,[{}],CUUR0000SA0
2,2021,February,263.014,[{}],CUUR0000SA0
3,2021,January,261.582,[{}],CUUR0000SA0
4,2020,December,260.474,[{}],CUUR0000SA0
...,...,...,...,...,...
1152,2002,May,433,[{}],JTU300000000000000TSL
1153,2002,April,491,[{}],JTU300000000000000TSL
1154,2002,March,397,[{}],JTU300000000000000TSL
1155,2002,February,415,[{}],JTU300000000000000TSL


In [13]:
BLS_final.report.unique()

array(['CUUR0000SA0', 'CES3000000006', 'JTU300000000000000JOL',
       'JTU300000000000000HIL', 'JTU300000000000000TSL'], dtype=object)

In [14]:
BLS_final['report'] = BLS_final['report'].replace('JTU300000000000000HIL', 'Manufacturing Hires').replace('CUUR0000SA0', 'CPI') \
.replace('CES3000000006','Manufacturing Employed').replace('JTU300000000000000JOL', 'Manufacturing Job Openings') \
.replace('JTU300000000000000TSL','Manufacturing_Separations')

BLS_final

#['JTU300000000000000TSL','Manufacturing_Separations']

,year,month,value,footnotes,report
0,2021,April,267.054,[{}],CPI
1,2021,March,264.877,[{}],CPI
2,2021,February,263.014,[{}],CPI
3,2021,January,261.582,[{}],CPI
4,2020,December,260.474,[{}],CPI
...,...,...,...,...,...
1152,2002,May,433,[{}],Manufacturing_Separations
1153,2002,April,491,[{}],Manufacturing_Separations
1154,2002,March,397,[{}],Manufacturing_Separations
1155,2002,February,415,[{}],Manufacturing_Separations


In [15]:
BLS_final['report'].unique()

array(['CPI', 'Manufacturing Employed', 'Manufacturing Job Openings',
       'Manufacturing Hires', 'Manufacturing_Separations'], dtype=object)

In [26]:
BLS_data = BLS_final.to_csv('data/BLS_data.csv')

## Ignore all this... original prettytable series

In [16]:
# print(MSeparations)
# # JTU300000000000000TSL = Manufacturing Separations 

In [17]:
# type(MSeparations)

In [18]:
#x['series id'] = x['series id'].replace('JTU300000000000000TSL','Manufacturing_Separations')

Note that the output only produces the last series ID in the code.  Will need to run a separate job for each desired Series ID

In [19]:
# # JTU300000000000000HIL = Manufacturing Hires 
# headers = {'Content-type': 'application/json'}
# data = json.dumps({"seriesid": ['CUUR0000SA0', 'CES3000000006', 'JTU300000000000000JOL', 
#                                 'JTU300000000000000HIL'],
#                                 "startyear":"2002",
#                                 "endyear":"2021", 
#                                 "registrationkey":"92eed6734bf5495f88972d9ba4106350"
#                                 #"calculations":"true"   #this does not appear to work
#                   })
# p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
# json_data = json.loads(p.text)
# for series in json_data['Results']['series']:
#     MHires=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
#     seriesId = series['seriesID']
#     for item in series['data']:
#         year = item['year']
#         period = item['period']
#         value = item['value']
#         footnotes=""
#         for footnote in item['footnotes']:
#             if footnote:
#                 footnotes = footnotes + footnote['text'] + ','
#         if 'M01' <= period <= 'M12':
#             MHires.add_row([seriesId,year,period,value,footnotes[0:-1]])
#     output = open(seriesId + '.txt','w')
#     output.write (MHires.get_string())
#     output.close()

In [20]:
#print(MHires)

In [21]:
# #JTU300000000000000JOL = Manufacturing all areas Job Openings
# headers = {'Content-type': 'application/json'}
# data = json.dumps({"seriesid": ['CUUR0000SA0', 'CES3000000006', 'JTU300000000000000JOL' 
#                                 ],
#                                 "startyear":"2002",
#                                 "endyear":"2021", 
#                                 "registrationkey":"92eed6734bf5495f88972d9ba4106350"
#                                 #"calculations":"true"
#                   })
# p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
# json_data = json.loads(p.text)
# for series in json_data['Results']['series']:
#     MJobs=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
#     seriesId = series['seriesID']
#     for item in series['data']:
#         year = item['year']
#         period = item['period']
#         value = item['value']
#         footnotes=""
#         for footnote in item['footnotes']:
#             if footnote:
#                 footnotes = footnotes + footnote['text'] + ','
#         if 'M01' <= period <= 'M12':
#             MJobs.add_row([seriesId,year,period,value,footnotes[0:-1]])
#     output = open(seriesId + '.txt','w')
#     output.write (MJobs.get_string())
#     output.close()

In [22]:
# print(MJobs)

In [23]:
# #CES3000000006 = Manufacturing Employed
# headers = {'Content-type': 'application/json'}
# data = json.dumps({"seriesid": ['CUUR0000SA0', 'CES3000000006' 
#                                 ],
#                                 "startyear":"2002",
#                                 "endyear":"2021", 
#                                 "registrationkey":"92eed6734bf5495f88972d9ba4106350"
#                                 #"calculations":"true"
#                   })
# p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
# json_data = json.loads(p.text)
# for series in json_data['Results']['series']:
#     MWorkers=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
#     seriesId = series['seriesID']
#     for item in series['data']:
#         year = item['year']
#         period = item['period']
#         value = item['value']
#         footnotes=""
#         for footnote in item['footnotes']:
#             if footnote:
#                 footnotes = footnotes + footnote['text'] + ','
#         if 'M01' <= period <= 'M12':
#             MWorkers.add_row([seriesId,year,period,value,footnotes[0:-1]])
#     output = open(seriesId + '.txt','w')
#     output.write (MWorkers.get_string())
#     output.close()

In [24]:
#print(MWorkers)